In [18]:
import os
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import rasterio
from rasterio.plot import show 
import numpy as np 
import leafmap 
from mpl_toolkits.axes_grid1 import make_axes_locatable 
from matplotlib.colors import LinearSegmentedColormap
import geopandas as gpd 
from rasterio.mask import mask 
import ipywidgets as widgets
from IPython.display import display
from osgeo import gdal, osr
import pandas as pd 
from shapely.geometry import Polygon
import datacube

In [19]:
date_prisma_w = widgets.Dropdown(
    options=['2023-02-09', '2023-03-22', '2023-04-08', '2023-06-17', '2023-07-10', '2023-08-08'],
    value='2023-02-09',
    description='PRISMA date:',
    disabled=False,
    layout={'width': 'max-content'},
    style = {'description_width': 'initial'}
)
date_prisma_w

Dropdown(description='PRISMA date:', layout=Layout(width='max-content'), options=('2023-02-09', '2023-03-22', …

In [20]:
date_s2_w = widgets.Dropdown(
    options=['2023-02-10', '2023-03-22', '2023-04-26', '2023-06-25', '2023-07-10', '2023-08-19'],
    value='2023-02-10',
    description='Sentinel-2 date:',
    disabled=False,
    layout={'width': 'max-content'},
    style = {'description_width': 'initial'}
)
date_s2_w

Dropdown(description='Sentinel-2 date:', layout=Layout(width='max-content'), options=('2023-02-10', '2023-03-2…

In [21]:
classification_method_w = widgets.RadioButtons(
    options=['RandomForest', 'XGBoost', 'AdaBoost', 'GradientBoost'],
    value='RandomForest', 
    layout={'width': 'max-content'}, 
    description='Classifier:',
    disabled=False
)
classification_method_w

RadioButtons(description='Classifier:', layout=Layout(width='max-content'), options=('RandomForest', 'XGBoost'…

In [22]:
Sensor = widgets.RadioButtons(
    options=['PRISMA', 'Sentinel-2'],
    value='PRISMA',  # Default to 'PRISMA'
    layout={'width': 'max-content'},
    description='Select Platform:',
    disabled=False
)
Sensor

RadioButtons(description='Select Platform:', layout=Layout(width='max-content'), options=('PRISMA', 'Sentinel-…

In [23]:
Sensor.value

'PRISMA'

In [24]:
# Get the current directory where the notebook is located
current_directory = os.getcwd()
CLIMAMI_folder_path = '/home/asi/data/meteo/climami'
# Get a list of GeoTIFF file names in the folder
geotiff_files = [file for file in os.listdir(CLIMAMI_folder_path) if file.endswith('.tif')]

# Create a dropdown widget to display the file names
file_dropdown = widgets.Dropdown(
    options=geotiff_files,
    description='Select a GeoTIFF file:'
)
file_dropdown

Dropdown(description='Select a GeoTIFF file:', options=('raster_minima_INVERNO_UHI_2200.tif', 'raster_minima_I…

In [25]:
dc = datacube.Datacube(app = 'my_app', config = '/home/asi/datacube_conf/datacube.conf')
list_of_products = dc.list_products()
list_of_products

,name,description,license,default_crs,default_resolution
name,,,,,
PRISMA_Full_Bands,PRISMA_Full_Bands,PRISMA imagery containing all bands,None,None,None
S2_Full_Bands,S2_Full_Bands,Sentinel-2 imagery containing all bands,None,None,None
UCP,UCP,Urban Canopy Parameters,None,None,None
classified_images,classified_images,Classified images using scikit-learn Random Fo...,None,None,None
climami,climami,raster data retrived from CLIMAMI Project (Oss...,None,None,None
training_samples,training_samples,Training Samples for LCZ classification,None,None,None


In [26]:
product_PRISMA = 'PRISMA_Full_Bands'
ds_PRISMA = dc.load(product = product_PRISMA, output_crs="epsg:32632", resolution=[30,30])
ds_PRISMA

<xarray.Dataset>
Dimensions:      (time: 6, y: 1207, x: 1232)
Coordinates:
  * time         (time) datetime64[ns] 2023-02-09T10:30:00 ... 2023-08-08T10:...
  * y            (y) float64 5.014e+06 5.014e+06 5.014e+06 ... 5.05e+06 5.05e+06
  * x            (x) float64 4.888e+05 4.889e+05 ... 5.257e+05 5.258e+05
    spatial_ref  int32 32632
Data variables: (12/239)
    band1        (time, y, x) float32 nan 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    band2        (time, y, x) float32 nan 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    band3        (time, y, x) float32 nan 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    band4        (time, y, x) float32 nan 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    band5        (time, y, x) float32 nan 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    band6        (time, y, x) float32 nan 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    ...           ...
    band234      (time, y, x) float32 nan 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    band235      (time, y, x) float32 nan 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    band236      (time, y, x) float32 nan 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    band237      (time, y, x) float32 nan 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    band238      (time, y, x) float32 nan 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    band239      (time, y, x) float32 nan 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
Attributes:
    crs:           EPSG:32632
    grid_mapping:  spatial_ref

In [ ]:
def process_plots(sensor, prisma_date, s2_date, classification_method):
    current_directory= os.getcwd()
    output_directory_plots = os.path.join(current_directory, 'PRISMA_CLIMAMI_colored_plot')
    os.makedirs( output_directory_plots, exist_ok = True)
    output_directory_clipped = os.path.join(current_directory, 'clipped_Sentinel_CLIMAMI')
    os.makedirs(  output_directory_clipped, exist_ok = True)
    sel_S2image_date=s2_date
    
    prisma_path = 'classified_images/'+f'classified_{classification_method}_{prisma_date.replace("-","")}_medianfilter_30m.tif'
    sentinel2_path= 'classified_images/'+f'classified_{classification_method}_{s2_date.replace("-","")}_medianfilter_20m.tif'
    extent_image_path= 'classified_images/'f'classified_{classification_method}_20230209_medianfilter_30m.tif'
    if sensor =='PRISMA':
        folder_path= prisma_path
        with rasterio.open(folder_path)as dataset:
            image_data = dataset.read()
            transform = dataset.transform
            legend = {
                   2: ['Compact mid-rise', '#D10000'],
                   3: ['Compact low-rise', '#CD0000'],
                   5: ['Open mid-rise', '#FF6600'],
                   6: ['Open low-rise', '#FF9955'],
                   8: ['Large low-rise', '#BCBCBC'],
                   101: ['Dense trees', '#006A00'],
                   102: ['Scattered trees', '#00AA00'],
                   104: ['Low plants', '#B9DB79'],
                   105: ['Bare rock or paved', '#545454'],
                   106: ['Bare soil or sand', '#FBF7AF'],
                   107: ['Water', '#6A6AFF']
            } 
            
            cmap_colors = [legend[key][1] for key in legend.keys()]
            cmap= plt.cm.colors.ListedColormap(cmap_colors, name= 'LCZ classes colormap')
            bounds = [int(key) for key in legend.keys()]
            bounds.append(bounds[-1]+1)
            norm = colors.BoundaryNorm(bounds, cmap.N)
            fig2 = plt.figure(figsize=(14,14))
            ax = fig2.add_subplot(1,1,1)
            plt.title(f'{sensor} {prisma_date} Classifed LCZ image')
            
            
            
            # Turn off the axes frame and labels
            ax.axis('off')
            ax.set_xticks([])
            ax.set_yticks([])
    
            # Add a legend  class colors
            labels = [legend[i][0] for i in legend.keys()]
            handles = [plt.Rectangle((0, 0), 1, 1, color=legend[key][1]) for key in list(legend.keys())]
            legend = plt.legend(handles, labels, title='LCZ Classes', bbox_to_anchor=(0.99, 1), loc='upper left')
            plt.setp(legend.get_title(), fontsize='12')  # Adjust the font size of the legend title
            show(image_data, cmap=cmap, interpolation='none',norm=norm);
            save_lczplotPRI_path = os.path.join(output_directory_plots,f'classified_{classification_method}_{prisma_date.replace("-","")}_medianfilter_30m_colored.png' )
            fig2.savefig(save_lczplotPRI_path)
            print(f"Plot saved to: {save_lczplotPRI_path}")
            
            
            
            
            
            
        
    elif sensor == 'Sentinel-2':
        folder_path= sentinel2_path
        with rasterio.open(folder_path)as src:
            with rasterio.open(extent_image_path, 'r+')as dataset:
                image_data = dataset.read()
                transform = dataset.transform
                vmin, vmax = np.nanmin(image_data), np.nanmax(image_data)
                image_data[image_data==0] = np.nan
                dataset.write(image_data)
                
                bounds = dataset.bounds
                min_x, min_y, max_x, max_y = bounds
                
                polygon_geom = Polygon([(min_x, min_y), (min_x, max_y), (max_x, max_y), (max_x, min_y)])
                polygon = gpd.GeoDataFrame(index=[0], crs=dataset.crs, geometry=[polygon_geom])
                shapes = polygon.geometry 
            clipped_data, clipped_transform = mask(src, shapes, crop=True)
            # Update the metadata
            meta = src.meta.copy()
            meta.update({
               'driver': 'GTiff',
               'height': clipped_data.shape[1],
               'width': clipped_data.shape[2],
               'transform': clipped_transform
             })
            save_path = os.path.join(output_directory_clipped,'classified_' + classification_method_w.value + '_' + sel_S2image_date.replace('-', '') + '_medianfilter_20m_clipped.tif' )
            # Save the clipped image 
            with rasterio.open(save_path, 'w', **meta) as dst:
                dst.write(clipped_data)
  
                print("Clipping completed.")
        folder_path = os.path.join(output_directory_clipped,'classified_' + classification_method_w.value + '_' + sel_S2image_date.replace('-', '') + '_medianfilter_20m_clipped.tif' )
        
    
        legend = {
                   2: ['Compact mid-rise', '#D10000'],
                   3: ['Compact low-rise', '#CD0000'],
                   5: ['Open mid-rise', '#FF6600'],
                   6: ['Open low-rise', '#FF9955'],
                   8: ['Large low-rise', '#BCBCBC'],
                   101: ['Dense trees', '#006A00'],
                   102: ['Scattered trees', '#00AA00'],
                   104: ['Low plants', '#B9DB79'],
                   105: ['Bare rock or paved', '#545454'],
                   106: ['Bare soil or sand', '#FBF7AF'],
                   107: ['Water', '#6A6AFF']
            } 
            
        cmap_colors = [legend[key][1] for key in legend.keys()]
        cmap= plt.cm.colors.ListedColormap(cmap_colors, name= 'LCZ classes colormap')
        bounds = [int(key) for key in legend.keys()]
        bounds.append(bounds[-1]+1)
        norm = colors.BoundaryNorm(bounds, cmap.N)
        fig1 = plt.figure(figsize=(14,14))
        ax = fig1.add_subplot(1,1,1)
        plt.title(f'{sensor} {s2_date} Classifed LCZ image')
        # Turn off the axes frame and labels
        ax.axis('off')
        ax.set_xticks([])
        ax.set_yticks([])
    
        # Add a legend  class colors
        labels = [legend[i][0] for i in legend.keys()]
        handles = [plt.Rectangle((0, 0), 1, 1, color=legend[key][1]) for key in list(legend.keys())]
        legend = plt.legend(handles, labels, title='LCZ Classes', bbox_to_anchor=(0.99, 1), loc='upper left')
        plt.setp(legend.get_title(), fontsize='12')  # Adjust the font size of the legend title
            
        show(clipped_data, cmap=cmap, interpolation='none',norm=norm);
        plt.tight_layout()
        save_lczplotS2_path = os.path.join(output_directory_plots,f'classified_{classification_method}_{s2_date.replace("-","")}_medianfilter_20m_colored.png' )
        fig1.savefig(save_lczplotS2_path)
        print(f"Plot saved to: {save_lczplotS2_path}")
            
    else:
        folder_path=none
    
    
            